In [1]:
from pyspark.sql import SparkSession

spark = (SparkSession
        .builder
        .appName("SparkSQLExampleApp")
        .getOrCreate())

In [2]:
csv_file = 'C:/0. Modeling/1. Modeling Code/pyspark_dataset/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/departuredelays.csv'

df = (spark.read.format('csv')
     .option('infraSchema', "true")
     .option("header","true")
     .load(csv_file))

In [3]:
# 임시뷰 생성
df.createOrReplaceTempView("us_delay_fligts_tbl")

In [4]:
df.show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|    6|     602|   ABE|        ATL|
|01020600|   -8|     369|   ABE|        DTW|
|01021245|   -2|     602|   ABE|        ATL|
|01020605|   -4|     602|   ABE|        ATL|
|01031245|   -4|     602|   ABE|        ATL|
|01030605|    0|     602|   ABE|        ATL|
|01041243|   10|     602|   ABE|        ATL|
|01040605|   28|     602|   ABE|        ATL|
|01051245|   88|     602|   ABE|        ATL|
|01050605|    9|     602|   ABE|        ATL|
|01061215|   -6|     602|   ABE|        ATL|
|01061725|   69|     602|   ABE|        ATL|
|01061230|    0|     369|   ABE|        DTW|
|01060625|   -3|     602|   ABE|        ATL|
|01070600|    0|     369|   ABE|        DTW|
|01071725|    0|     602|   ABE|        ATL|
|01071230|    0|     369|   ABE|        DTW|
|01070625|    0|     602|   ABE|        ATL|
|01071219|    0|     569|   ABE|        ORD|
|01080600|

# p.90~ 예제

## SQL코드 - 비행거리가 1000이상인 항공편

In [5]:
spark.sql("""SELECT distance, origin, destination
FROM us_delay_fligts_tbl WHERE distance > 1000
ORDER BY distance DESC""").show(10)

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows



### spark.sql 인터페이스 사용 - 비행거리가 1000이상인 항공편

In [6]:
from pyspark.sql.functions import col, desc
(df.select("distance","origin","destination")
    .where(col("distance")>1000)
    .orderBy("distance", ascending = False).show(10))

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows



## SQL코드 - SFO-ORD 사이에서 연착이 120분 이상 항공편

In [7]:
spark.sql("""SELECT date, delay, origin, destination
FROM us_delay_fligts_tbl
WHERE delay > 120 AND ORIGIN = 'SFO' AND DESTINATION = 'ORD'
ORDER BY delay DESC""").show(10)

+--------+-----+------+-----------+
|    date|delay|origin|destination|
+--------+-----+------+-----------+
|01031755|  396|   SFO|        ORD|
|01022330|  326|   SFO|        ORD|
|01051205|  320|   SFO|        ORD|
|01190925|  297|   SFO|        ORD|
|02171115|  296|   SFO|        ORD|
|01071040|  279|   SFO|        ORD|
|01051550|  274|   SFO|        ORD|
|03120730|  266|   SFO|        ORD|
|01261104|  258|   SFO|        ORD|
|01161210|  225|   SFO|        ORD|
+--------+-----+------+-----------+
only showing top 10 rows



### spark.sql 인터페이스 사용 -  SFO-ORD 사이에서 연착이 120분 이상 항공편

In [8]:
(df.select("date","delay","origin","destination")
    .where((col("delay")>120) & (col("origin")=='SFO') & (col('destination') == 'ORD'))
    .orderBy("delay", ascending = False).show(10))

+--------+-----+------+-----------+
|    date|delay|origin|destination|
+--------+-----+------+-----------+
|01031755|  396|   SFO|        ORD|
|01022330|  326|   SFO|        ORD|
|01051205|  320|   SFO|        ORD|
|01190925|  297|   SFO|        ORD|
|02171115|  296|   SFO|        ORD|
|01071040|  279|   SFO|        ORD|
|01051550|  274|   SFO|        ORD|
|03120730|  266|   SFO|        ORD|
|01261104|  258|   SFO|        ORD|
|01161210|  225|   SFO|        ORD|
+--------+-----+------+-----------+
only showing top 10 rows



## SQL코드 - 지연에 대한 표시를 레이블로 지정

In [9]:
spark.sql("""SELECT delay, origin, destination,
            CASE
                WHEN delay > 360 THEN 'Very Long Delays'
                WHEN delay >= 120 AND delay <=360 THEN 'Long Delays'
                WHEN delay >= 60 AND delay <= 120 THEN 'Short Delays'
                WHEN delay > 0 AND delay < 60 THEN 'Tolerable Delays'
                WHEN delay = 0 THEN 'No Delays'
                ELSE 'Early'
            END AS Flight_Delays
            FROM us_delay_fligts_tbl
            ORDER BY origin, delay DESC""").show(10)

+-----+------+-----------+----------------+
|delay|origin|destination|   Flight_Delays|
+-----+------+-----------+----------------+
|   92|   ABE|        ORD|    Short Delays|
|   91|   ABE|        DTW|    Short Delays|
|    9|   ABE|        ORD|Tolerable Delays|
|    9|   ABE|        ATL|Tolerable Delays|
|    9|   ABE|        ATL|Tolerable Delays|
|    9|   ABE|        ATL|Tolerable Delays|
|    9|   ABE|        ATL|Tolerable Delays|
|    9|   ABE|        ATL|Tolerable Delays|
|   89|   ABE|        DTW|    Short Delays|
|   88|   ABE|        ATL|    Short Delays|
+-----+------+-----------+----------------+
only showing top 10 rows



### spark.sql 인터페이스 사용 - 지연에 대한 표시를 레이블로 지정

In [10]:
from pyspark.sql.functions import *

In [11]:
(df.select("delay","origin","destination")
    .withColumn("flights_delays", when(col('delay') > 360 ,'Very Long Delays')
    .when((col("delay") > 120) & (col("delay") <=360), 'Long Delays')
    .when((col("delay") > 60) & (col("delay") <= 120), 'Short Delays')
    .when((col("delay") > 0) & (col("delay") <= 60 ), 'Tolerable Delays')
    .when(col("delay") == 0, 'No delays')
    .otherwise("Early"))
    .orderBy("origin", "delay", ascending = False).show(10))

+-----+------+-----------+----------------+
|delay|origin|destination|  flights_delays|
+-----+------+-----------+----------------+
|   97|   YUM|        LAX|    Short Delays|
|   97|   YUM|        LAX|    Short Delays|
|   95|   YUM|        PHX|    Short Delays|
|    9|   YUM|        LAX|Tolerable Delays|
|    9|   YUM|        LAX|Tolerable Delays|
|    9|   YUM|        PHX|Tolerable Delays|
|    9|   YUM|        PHX|Tolerable Delays|
|    9|   YUM|        LAX|Tolerable Delays|
|    9|   YUM|        LAX|Tolerable Delays|
|   82|   YUM|        LAX|    Short Delays|
+-----+------+-----------+----------------+
only showing top 10 rows



# SQL 테이블과 뷰

In [5]:
spark.sql("CREATE DATABASE learn_spark_db")

DataFrame[]

In [6]:
spark.sql("USE learn_spark_db")

DataFrame[]

## 관리형 테이블 생성

In [19]:
spark.sql("CREATE TABLE managed_us_delay_flights_tbl(date STRING, delay INT, distance INT, origin STRING, destination STRING)")

AnalysisException: Hive support is required to CREATE Hive TABLE (AS SELECT);
'CreateTable `learn_spark_db`.`managed_us_delay_flights_tbl`, org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe, ErrorIfExists


In [8]:
csv_file = 'C:/0. Modeling/1. Modeling Code/LearningSparkV2-master/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/departuredelays.csv'

In [9]:
schema = "date STRING, delay INT, distance INT, origin STRING, destination STRING"
flights_df = spark.read.csv(csv_file, schema = schema)

In [10]:
flights_df.show(10)

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|    date| null|    null|origin|destination|
|01011245|    6|     602|   ABE|        ATL|
|01020600|   -8|     369|   ABE|        DTW|
|01021245|   -2|     602|   ABE|        ATL|
|01020605|   -4|     602|   ABE|        ATL|
|01031245|   -4|     602|   ABE|        ATL|
|01030605|    0|     602|   ABE|        ATL|
|01041243|   10|     602|   ABE|        ATL|
|01040605|   28|     602|   ABE|        ATL|
|01051245|   88|     602|   ABE|        ATL|
+--------+-----+--------+------+-----------+
only showing top 10 rows



In [11]:
flights_df.write.saveAsTable("managed_us_delay_flights_tbl")

## 비관리형 테이블 생성하기 

In [14]:
spark.sql("""
CREATE TABLE us_delay_flights_tbl (
    date STRING,
    delay INT,
    distance INT,
    origin STRING,
    destination STRING)
USING csv OPTIONS (PATH 'C:/0. Modeling/1. Modeling Code/LearningSparkV2-master/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/departuredelays.csv')
""")

DataFrame[]

In [16]:
(flights_df
    .write
    .option("path",'/tmp/data/us_flights_delay')
    .saveAsTable("us_delay_flights_tbl2"))

# 파케이

In [20]:
file = """C:/0. Modeling/1. Modeling Code/pyspark_dataset/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/parquet/2010-summary.parquet/"""

In [21]:
df = spark.read.format("parquet").load(file)

In [22]:
df.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



## 파케이 파일을 Spark SQL 테이블로 읽기

In [23]:
spark.sql("""CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl
                USING parquet
                OPTIONS (
                path "C:/0. Modeling/1. Modeling Code/pyspark_dataset/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/parquet/2010-summary.parquet/")""")

DataFrame[]

In [24]:
spark.sql("SELECT * FROM us_delay_flights_tbl").show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|    1|
|       United States|            Ireland|  264|
|       United States|              India|   69|
|               Egypt|      United States|   24|
|   Equatorial Guinea|      United States|    1|
|       United States|          Singapore|   25|
|       United States|            Grenada|   54|
|          Costa Rica|      United States|  477|
|             Senegal|      United States|   29|
|       United States|   Marshall Islands|   44|
|              Guyana|      United States|   17|
|       United States|       Sint Maarten|   53|
|               Malta|      United States|    1|
|             Bolivia|      United States|   46|
|            Anguilla|      United States|   21|
|Turks and Caicos ...|      United States|  136|
|       United States|        Afghanistan|    2|
|Saint Vincent and..

## 데이터 프레임을 파케이 파일로 쓰기

In [25]:
(df.write.format("parquet")
    .mode("overwrite")
    .option("compression", "snappy")
           .save("tmp/data/parquet/df_parquet"))

In [26]:
(df.write
    .mode("overwrite")
    .saveAsTable("us_delay_flights_tbl"))

# JSON

## JSON 파일을 데이터 프레임으로 읽기

In [27]:
file = "C:/0. Modeling/1. Modeling Code/pyspark_dataset/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/json/*"
df = spark.read.format("json").load(file)

In [28]:
df.show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

### 스파크 SQL 테이블로 JSON 파일 읽기

In [29]:
spark.sql("""CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl
                USING json
                OPTIONS (
                path "C:/0. Modeling/1. Modeling Code/pyspark_dataset/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/json/*")
""")

DataFrame[]

In [30]:
spark.sql("SELECT * FROM us_delay_flights_tbl").show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows



## 데이터 프레임을 JSON 파일로 쓰기

In [31]:
(df.write.format("json")
         .mode("overwrite")
         .option("compression","snappy")
         .save("/tmp/data/json/df_json"))

# CSV

## CSV 파일을 데이터 프레임으로 읽기

In [2]:
file = "C:/0. Modeling/1. Modeling Code/pyspark_dataset/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/csv/*"

schema = "DEST_COUNTRY_NAME STRING, ORIGIN_COUNTRY_NAME STRING, count INT"

df = (spark.read.format("csv")
     .option("header", "true")
     .schema(schema)
     .option("mode","FAILFAST")
     .option('nullValue',"")
     .load(file))

In [3]:
df.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



## CSV 파일을 스파크 SQL 테이블로 읽기

In [5]:
spark.sql("""CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl
             USING csv
         OPTIONS (
         path "C:/0. Modeling/1. Modeling Code/pyspark_dataset/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/csv/*",
         header "true",
         inferSchema "true",
         mode "FAILFAST")
         """)

DataFrame[]

In [6]:
spark.sql("SELECT * FROM us_delay_flights_tbl").show(10)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
|    United States|          Singapore|   25|
|    United States|            Grenada|   54|
|       Costa Rica|      United States|  477|
|          Senegal|      United States|   29|
|    United States|   Marshall Islands|   44|
+-----------------+-------------------+-----+
only showing top 10 rows



## 데이터 프레임을 CSV 파일로 쓰기

In [7]:
df.write.format("csv").mode("overwrite").save("/tmp/data/csv/df_csv")

# 에이브로

In [8]:
df = (spark.read.format("avro")
     .load("C:/0. Modeling/1. Modeling Code/pyspark_dataset/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/avro/*"))

AnalysisException:  Failed to find data source: avro. Avro is built-in but external data source module since Spark 2.4. Please deploy the application as per the deployment section of "Apache Avro Data Source Guide".        

# ORC

In [9]:
file = "C:/0. Modeling/1. Modeling Code/pyspark_dataset/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/orc/*"

df = spark.read.format("orc").option("path", file).load()

In [10]:
df.show(10,False)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |1    |
|United States    |Ireland            |264  |
|United States    |India              |69   |
|Egypt            |United States      |24   |
|Equatorial Guinea|United States      |1    |
|United States    |Singapore          |25   |
|United States    |Grenada            |54   |
|Costa Rica       |United States      |477  |
|Senegal          |United States      |29   |
|United States    |Marshall Islands   |44   |
+-----------------+-------------------+-----+
only showing top 10 rows



In [11]:
spark.sql("""CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl
             USING orc
         OPTIONS (
         path "C:/0. Modeling/1. Modeling Code/pyspark_dataset/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/orc/*")
         """)

DataFrame[]

In [12]:
spark.sql("SELECT * FROM us_delay_flights_tbl").show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|    1|
|       United States|            Ireland|  264|
|       United States|              India|   69|
|               Egypt|      United States|   24|
|   Equatorial Guinea|      United States|    1|
|       United States|          Singapore|   25|
|       United States|            Grenada|   54|
|          Costa Rica|      United States|  477|
|             Senegal|      United States|   29|
|       United States|   Marshall Islands|   44|
|              Guyana|      United States|   17|
|       United States|       Sint Maarten|   53|
|               Malta|      United States|    1|
|             Bolivia|      United States|   46|
|            Anguilla|      United States|   21|
|Turks and Caicos ...|      United States|  136|
|       United States|        Afghanistan|    2|
|Saint Vincent and..

In [15]:
(df.write.format("orc")
  .mode("overwrite")
  .option("compression", "snappy")
  .save("/tmp/data/orc/df_orc"))

# 이미지

In [16]:
from pyspark.ml import image

image_dir = "C:/0. Modeling/1. Modeling Code/pyspark_dataset/LearningSparkV2-master/databricks-datasets/learning-spark-v2/cctvVideos/train_images/"
images_df = spark.read.format("image").load(image_dir)
images_df.printSchema()

C:\Users\USER\Anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = true)
 |    |-- width: integer (nullable = true)
 |    |-- nChannels: integer (nullable = true)
 |    |-- mode: integer (nullable = true)
 |    |-- data: binary (nullable = true)
 |-- label: integer (nullable = true)



In [23]:
images_df.select("image.height", "image.width","image.nChannels","image.mode","label").show(5, truncate = False)

Py4JJavaError: An error occurred while calling o112.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 10.0 failed 1 times, most recent failure: Lost task 0.0 in stage 10.0 (TID 20) (R14286N01.lotterental.net executor driver): java.io.FileNotFoundException: 
File file:/C:/0.%20Modeling/1.%20Modeling%20Code/pyspark_dataset/LearningSparkV2-master/databricks-datasets/learning-spark-v2/cctvVideos/train_images/label=0/LeftBagframe0004.jpg does not exist

It is possible the underlying files have been updated. You can explicitly invalidate
the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by
recreating the Dataset/DataFrame involved.
       
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:661)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:212)
	

# 이진 파일

In [21]:
path = "C:/0. Modeling/1. Modeling Code/pyspark_dataset/LearningSparkV2-master/databricks-datasets/learning-spark-v2/cctvVideos/train_images/"
binary_files_df = (spark.read.format("binary_file")
                  .option("pathGlobFilter", "*.jpg")
                  .load(path))
binary_files_df.show(5)

Py4JJavaError: An error occurred while calling o96.load.
: java.lang.ClassNotFoundException: 
Failed to find data source: binary_file. Please find packages at
https://spark.apache.org/third-party-projects.html
       
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedToFindDataSourceError(QueryExecutionErrors.scala:587)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:675)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:725)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:207)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:185)
	at java.base/jdk.internal.reflect.DirectMethodHandleAccessor.invoke(DirectMethodHandleAccessor.java:104)
	at java.base/java.lang.reflect.Method.invoke(Method.java:578)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.

In [22]:
path = "C:/0. Modeling/1. Modeling Code/pyspark_dataset/LearningSparkV2-master/databricks-datasets/learning-spark-v2/cctvVideos/train_images/"
binary_files_df = (spark.read.format("binary_file")
                  .option("pathGlobFilter", "*.jpg")
                  .option("recursiveFileLookup","true")
                  .load(path))
binary_files_df.show(5)

Py4JJavaError: An error occurred while calling o102.load.
: java.lang.ClassNotFoundException: 
Failed to find data source: binary_file. Please find packages at
https://spark.apache.org/third-party-projects.html
       
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedToFindDataSourceError(QueryExecutionErrors.scala:587)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:675)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:725)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:207)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:185)
	at java.base/jdk.internal.reflect.DirectMethodHandleAccessor.invoke(DirectMethodHandleAccessor.java:104)
	at java.base/java.lang.reflect.Method.invoke(Method.java:578)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j